Install Pipenv
Check the version of pipenv I installed
Use --version to find out

In [ ]:
%pip install pipenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
!pipenv --version

pipenv, version 2024.2.0


Use Pipenv to install Scikit-Learn version 1.5.2
Check The first hash for scikit-learn I get in Pipfile.l

In [3]:
!pipenv install scikit-learn==1.5.2

Creating a Pipfile for this project...


Pipfile.lock not found, creating...
Locking [packages] dependencies...
Locking [dev-packages] dependencies...
Updated Pipfile.lock (702ad05de9bc9de99a4807c8dde1686f31e0041d7b5f6f6b74861195a52110f5)!
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.
Installing scikit-learn==1.5.2...
✔ Installation Succeeded
Building requirements...
Resolving dependencies....
✔ Success! Locking packages...
⠼ Locking packages...
Building requirements...
Resolving dependencies....
✔ Success! Locking packages...
⠴ Locking packages...
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.
Installing dependencies from Pipfile.lock (6757b9)...
All dependencies are now up-to-date!
Installing dependencies from Pipfile.lock (6757b9)...


In [4]:
!cat /workspaces/ML-ZoomCamp2024-Homework/5-deploying-models/Pip/Pipfile.lock

{
    "_meta": {
        "hash": {
            "sha256": "adb15e5c5a21f13e221698df1bb36dc8d3bee16bb778fdfcad74b126616757b9"
        },
        "pipfile-spec": 6,
        "requires": {
            "python_version": "3.12"
        },
        "sources": [
            {
                "name": "pypi",
                "url": "https://pypi.org/simple",
                "verify_ssl": true
            }
        ]
    },
    "default": {
        "joblib": {
            "hashes": [
                "sha256:06d478d5674cbc267e7496a410ee875abd68e4340feff4490bcb7afb88060ae6",
                "sha256:2382c5816b2636fbd20a09e0f4e9dad4736765fdfb7dca582943b9c1366b3f0e"
            ],
            "markers": "python_version >= '3.8'",
            "version": "==1.4.2"
        },
        "numpy": {
            "hashes": [
                "sha256:05b2d4e667895cc55e3ff2b56077e4c8a5604361fc21a042845ea3ad67465aa8",
                "sha256:12edb90831ff481f7ef5f6bc6431a9d74dc0e5ff401559a71e5e4611d4f2d466",
         

In [11]:
!wget https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/master/cohorts/2024/05-deployment/homework/model1.bin
!wget https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/master/cohorts/2024/05-deployment/homework/dv.bin

--2024-10-29 09:43:35--  https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/master/cohorts/2024/05-deployment/homework/model1.bin
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 850 [application/octet-stream]
Saving to: ‘model1.bin’

model1.bin          100%[===================>]     850  --.-KB/s    in 0s      

2024-10-29 09:43:36 (33.2 MB/s) - ‘model1.bin’ saved [850/850]

--2024-10-29 09:43:36--  https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/master/cohorts/2024/05-deployment/homework/dv.bin
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... con

In [5]:
!ls

Pip  Pipfile  Pipfile.lock  deployment.ipynb  dv.bin  model1.bin


Let's use these models!

A script for loading these models with pickle


In [6]:
import pickle

with open('dv.bin', 'rb') as f_in:
    dv = pickle.load(f_in)

with open('model1.bin', 'rb') as f_in:
    model = pickle.load(f_in)

client = {"job": "management", "duration": 400, "poutcome": "success"}

X = dv.transform([client])

probability = model.predict_proba(X)[0, 1]

print(f"Probability of subscription: {probability:.3f}")


Probability of subscription: 0.759


Now let's serve this model as a web service

Install Flask and gunicorn (or waitress, if you're on Windows)
Write Flask code for serving the model
Now score this client using requests

In [18]:
%pip install Flask 

Note: you may need to restart the kernel to use updated packages.


In [2]:
from flask import Flask, request, jsonify
import pickle
with open('dv.bin', 'rb') as f_in:
    dv = pickle.load(f_in)

with open('model1.bin', 'rb') as f_in:
    model = pickle.load(f_in)

app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():
    client = request.get_json() 
    X = dv.transform([client])  
    prob = model.predict_proba(X)[0, 1]  
    
    # Return the probability as a JSON response
    result = {"subscription_probability": prob}
    return jsonify(result)

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=8001, debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8001
 * Running on http://10.0.3.126:8001
Press CTRL+C to quit
 * Restarting with stat
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/codespace/.local/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/codespace/.local/lib/python3.12/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/home/codespace/.local/lib/python3.12/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/codespace/.local/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 692, in initialize
    self.init_sockets()
  File "/home/codespace/.local/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 331,

SystemExit: 1

/home/codespace/.local/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
!sudo apt update

Get:1 https://dl.yarnpkg.com/debian stable InRelease [17.1 kB]
Get:2 https://dl.yarnpkg.com/debian stable/main amd64 Packages [11.8 kB]       
Get:3 https://dl.yarnpkg.com/debian stable/main all Packages [11.8 kB]         
Get:4 https://repo.anaconda.com/pkgs/misc/debrepo/conda stable InRelease [3961 B]0m
Get:5 https://packages.microsoft.com/repos/microsoft-ubuntu-focal-prod focal InRelease [3632 B]
Get:6 https://repo.anaconda.com/pkgs/misc/debrepo/conda stable/main amd64 Packages [4557 B]
Get:7 https://packages.microsoft.com/repos/microsoft-ubuntu-focal-prod focal/main amd64 Packages [319 kB]
Get:8 http://archive.ubuntu.com/ubuntu focal InRelease [265 kB]                
Get:9 https://packages.microsoft.com/repos/microsoft-ubuntu-focal-prod focal/main all Packages [2938 B]
Get:10 http://security.ubuntu.com/ubuntu focal-security InRelease [128 kB]     
Get:12 http://security.ubuntu.com/ubuntu focal-security/multiverse amd64 Packages [30.9 kB]3m
Get:13 http://archive.ubuntu.com/ubuntu f

In [2]:
!sudo apt install docker.io

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Some packages could not be installed. This may mean that you have
requested an impossible situation or if you are using the unstable
distribution that some required packages have not yet been created
or been moved out of Incoming.
The following information may help to resolve the situation:

The following packages have unmet dependencies:
 docker.io : Depends: containerd (>= 1.2.6-0ubuntu1~)
E: Unable to correct problems, you have held broken packages.


In [15]:
!docker info

Client:
 Version:    27.0.3-1
 Context:    default
 Debug Mode: false
 Plugins:
  buildx: Docker Buildx (Docker Inc.)
    Version:  v0.17.1
    Path:     /usr/libexec/docker/cli-plugins/docker-buildx
  compose: Docker Compose (Docker Inc.)
    Version:  v2.29.7
    Path:     /usr/libexec/docker/cli-plugins/docker-compose

Server:
 Containers: 1
  Running: 1
  Paused: 0
  Stopped: 0
 Images: 1
 Server Version: 27.0.3-1
 Storage Driver: overlay2
  Backing Filesystem: extfs
  Supports d_type: true
  Using metacopy: false
  Native Overlay Diff: false
  userxattr: false
 Logging Driver: json-file
 Cgroup Driver: cgroupfs
 Cgroup Version: 2
 Plugins:
  Volume: local
  Network: bridge host ipvlan macvlan null overlay
  Log: awslogs fluentd gcplogs gelf journald json-file local splunk syslog
 Swarm: inactive
 Runtimes: io.containerd.runc.v2 runc
 Default Runtime: runc
 Init Binary: docker-init
 containerd version: 88c3d9bc5b5a193f40b7c14fa996d23532d6f956
 runc version: 2c9f5602f0ba3d9da1c25963

In [13]:
!sudo service start docker
!sudo systemctl enable docker
!docker --version

start: unrecognized service



"systemd" is not running in this container due to its overhead.
Use the "service" command to start services instead. e.g.: 

service --status-all
Docker version 27.0.3-1, build 7d4bcd863a4c863e650eed02a550dfeb98560b83


In [16]:
!docker pull svizor/zoomcamp-model:3.11.5-slim

3.11.5-slim: Pulling from svizor/zoomcamp-model
Digest: sha256:15d61790363f892dfdef55f47b78feed751cb59704d47ea911df0ef3e9300c06
Status: Image is up to date for svizor/zoomcamp-model:3.11.5-slim
docker.io/svizor/zoomcamp-model:3.11.5-slim


In [17]:
!docker run -it --rm -p 8001:8001 svizor/zoomcamp-model:3.11.5-slim

docker: Error response from daemon: driver failed programming external connectivity on endpoint keen_hugle (b727398e5d23fe0cb4257d352251752bb0961ad02e34c606a2b296cb56b94a1a): Bind for 0.0.0.0:8001 failed: port is already allocated.


In [11]:
!curl http://localhost:8001

curl: (56) Recv failure: Connection reset by peer
